In [4]:
import uproot
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import glob
batch = 2048
import matplotlib.pyplot as plt

In [5]:
l = glob.glob("/home/zaptos/Code/data/**/*.root", recursive=True)

In [6]:
branches = ["SP_x", "SP_y", "SP_z"]
pad_lenght = 3
n=3

for j, file in enumerate(l):
    inp = np.empty((0,3,3))
    tar = np.empty((0,3))
    print(file)
    with uproot.open(file) as f:
        coordinateTree = f["SPInfo"]
        coordinate_dict = coordinateTree.arrays(branches, library='np')
    c1 = [c for c in coordinate_dict[branches[0]] if len(c) > 0]
    c2 = [c for c in coordinate_dict[branches[1]] if len(c) > 0]
    c3 = [c for c in coordinate_dict[branches[2]] if len(c) > 0]


    for i in range(len(c1)):
        c1_arr = c1[i].reshape(-1,1) / 1015
        c2_arr = c2[i].reshape(-1,1) / 1015
        c3_arr = c3[i].reshape(-1,1) / 3500
        track = np.hstack((c1_arr, c2_arr, c3_arr))
        _, idx = np.unique(track, axis=0, return_index=True)
        track = track[np.sort(idx)]
        #print(track) 
        track = np.pad(track, ((0, pad_lenght),(0,0)))
        # Reshape into rolling window of n=3
        #print(track)
        track = np.lib.stride_tricks.sliding_window_view(track, (n,3)).reshape(-1,n,3)
        inp = np.vstack((inp,track[:-1]))
        tar = np.vstack((tar, np.array([t[-1] for t in track[1:]])))

    np.save(f"data/{j}_input.npy", inp)
    np.save(f"data/{j}_target.npy", tar)

/home/zaptos/Code/data/data_for_seeding/RDORootFiles/v3p1/user.sabidi.PhPy8EG_AZNLO_Ztautau.e8481_s4203_r14697.v3.1_EXT0/user.sabidi.37663841.EXT0._000003.CluserInfo.root
/home/zaptos/Code/data/data_for_seeding/RDORootFiles/v3p1/user.sabidi.PhPy8EG_AZNLO_Ztautau.e8481_s4203_r14697.v3.1_EXT0/user.sabidi.37663841.EXT0._000001.CluserInfo.root


KeyboardInterrupt: 